In [1]:
from ExpDerive.nlp.compiler.db import DB, loadColumns, loadFunctions, Func
import json

/Users/danielfisaha/exp_derive/env_exp_derive/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:

api_key = "5837c25c-f54b-4dfc-9e4a-f00b694c0f67"
environment = "us-west4-gcp"
db = DB(api_key=api_key, environment=environment)

In [3]:
# columns = loadColumns("/Users/danielfisaha/exp_derive/ExpDerive/data/epl_columns.json")

In [4]:
# db.addVariables(columns)

In [3]:
db.getVariable("xg")

{'dataType': 'float',
 'description': 'goals scored',
 'label': 'goals',
 'type': 'standard'}

In [ ]:
# db.createIndex("builtIn")
# db.createIndex("custom")
# db.createIndex("operator")

In [9]:
builtIns = loadFunctions("/Users/danielfisaha/exp_derive/ExpDerive/data/functions.json", "builtIn", "function")
customs = loadFunctions("/Users/danielfisaha/exp_derive/ExpDerive/data/new_funcs.json", "custom", "function")
operators = loadFunctions("/Users/danielfisaha/exp_derive/ExpDerive/data/operators.json", "operator", "operator")

# db.addFunctions(functions)

In [10]:
db.addFunctions(builtIns)
db.addFunctions(customs)
db.addFunctions(operators)

In [23]:
idx = db.get_index("name")

In [33]:
metadata = {
    "label": "multiplied by",
    "description": "multiplied by",
    "type": "builtIn"
}
vector = db.getEmbedding("multiplied by")
idx.upsert(
    vectors=[("multiplied by", vector, metadata)],
    namespace="operator",
)


{'upserted_count': 1}

In [63]:
db.getFunction("times")

{'description': 'times', 'label': 'times', 'type': 'standard'}

In [40]:
with open("/Users/danielfisaha/exp_derive/ExpDerive/training_data/stats_test.json", "r") as f:
    stats = json.load(f)

In [21]:
total = 0
correct = 0
for stat in stats:
    right = 0
    print("predicting for", stat["label"])
    predictions = (
        (stat["phrasing1"], db.getVariable(stat["phrasing1"])["label"]),
        (stat["phrasing2"], db.getVariable(stat["phrasing2"])["label"]),
        (stat["phrasing3"], db.getVariable(stat["phrasing3"])["label"])
    )
    for prediction in predictions:
        total += 1
        if prediction[1] == stat["label"]:
            correct += 1
            right += 1
    stat_accuracy = f"{right*100/3:.2f}%"
    total_accuracy = f"{correct*100/total:.2f}%"
    print(stat_accuracy, predictions)
    print("total accuracy:" + total_accuracy)
    


predicting for games_starts
100.00% (('Started matches', 'games_starts'), ('Games as starter', 'games_starts'), ('Starting lineup games', 'games_starts'))
total accuracy:100.00%
predicting for minutes
100.00% (('Played minutes', 'minutes'), ('Total game time', 'minutes'), ('Duration on field', 'minutes'))
total accuracy:100.00%
predicting for minutes_90s
100.00% (('90-minute games', 'minutes_90s'), ('Matches (90 min)', 'minutes_90s'), ('Games (90 min)', 'minutes_90s'))
total accuracy:100.00%
predicting for goals
100.00% (('Scored goals', 'goals'), ('Total goals scored', 'goals'), ('Goals netted', 'goals'))
total accuracy:100.00%
predicting for assists
66.67% (('Provided assists', 'assists'), ('Total assists made', 'goals_assists'), ('Assists given', 'assists'))
total accuracy:93.33%
predicting for goals_assists
66.67% (('Goals & assists', 'goals_assists'), ('Scoring involvement', 'goals'), ('Combined goals/assists', 'goals_assists'))
total accuracy:88.89%
predicting for goals_pens
33.3

In [3]:
with open("/Users/danielfisaha/exp_derive/ExpDerive/training_data/function_test.json", "r") as f:
    functions = json.load(f)
    operators = functions["operators"]
    functions = functions["functions"]

In [4]:
def testFunctions(functions, namespace):
    total = 0
    correct = 0
    for function in functions:
        right = 0
        print("predicting for", function["saytex"])
        predictions = (
            (function["phrasing1"], db.getFunction(function["phrasing1"], namespace)["label"]),
            (function["phrasing2"], db.getFunction(function["phrasing2"], namespace)["label"]),
            (function["phrasing3"], db.getFunction(function["phrasing3"], namespace)["label"])
        )
        for prediction in predictions:
            total += 1
            if prediction[1] == function["saytex"]:
                correct += 1
                right += 1
        function_accuracy = f"{right*100/3:.2f}%"
        total_accuracy = f"{correct*100/total:.2f}%"
        print(function_accuracy, predictions)
        print("total accuracy:" + total_accuracy)

In [5]:
testFunctions(operators, "operator")

predicting for plus
100.00% (('Addition', 'plus'), ('Sum', 'plus'), ('Plus', 'plus'))
total accuracy:100.00%
predicting for minus
100.00% (('Subtraction', 'minus'), ('Difference', 'minus'), ('Minus', 'minus'))
total accuracy:100.00%
predicting for multiplied by
0.00% (('Multiplication', 'times'), ('Times', 'plus'), ('Product', 'plus'))
total accuracy:66.67%
predicting for divided by
100.00% (('Division', 'divided by'), ('Divided by', 'divided by'), ('Quotient', 'divided by'))
total accuracy:75.00%
predicting for percent of
100.00% (('Percentage of', 'percent of'), ('Percent of', 'percent of'), ('Proportion', 'percent of'))
total accuracy:80.00%


In [6]:
testFunctions(functions, "builtIn")

predicting for natural logarithm
66.67% (('Natural log', 'natural logarithm'), ('ln function', 'natural logarithm'), ('Logarithm (base e)', 'logarithm'))
total accuracy:66.67%
predicting for greatest common divisor
100.00% (('GCD', 'greatest common divisor'), ('Greatest divisor', 'greatest common divisor'), ('Largest common factor', 'greatest common divisor'))
total accuracy:83.33%
predicting for square root
100.00% (('Square root', 'square root'), ('Root square', 'square root'), ('Sqrt function', 'square root'))
total accuracy:88.89%
predicting for logarithm
100.00% (('Log', 'logarithm'), ('Logarithm function', 'logarithm'), ('Logarithmic value', 'logarithm'))
total accuracy:91.67%
predicting for factorial
100.00% (('Factorial', 'factorial'), ('Fact function', 'factorial'), ('Product of integers', 'factorial'))
total accuracy:93.33%
predicting for absolute value
100.00% (('Absolute value', 'absolute value'), ('Abs function', 'absolute value'), ('Magnitude (abs)', 'absolute value'))
to